### Calculate $\delta R$ matrix from combinations (Sequential Version)

Steps are:
- Take $\phi$ and $\eta$ data arrays,starts and stops. 
- Get combinations indices in same manner as done earlier
- For each pair, calculate deltaR as $$\delta r = \sqrt{\delta \eta^{2} + \delta \phi^{2}}$$ where $$\delta \eta = \eta_{2} - \eta_{1}$$$$ \delta \phi = \phi_{2} - \phi_{1}$$
- Get the matrix $$\delta R = \{ \delta r_{ij}\} \quad \forall \: (i,j) \: \in \: combinations \: indices$$


In [1]:
import numpy
import pycuda.driver as cuda
from pycuda.compiler import *
import pycuda.autoinit
import math

In [2]:
# Adapted from gen-reco script by Jim

NUMEVENTS = 100      # exact number of events
AVENUMJETS = 10      # average number of jets per event
PHILOW = -numpy.pi   # bounding box of phi (azimuthal angle) and eta (~polar angle)
PHIHIGH = numpy.pi
ETALOW = -5
ETAHIGH = 5
ERRPHI = 0.01        # detector resolution
ERRETA = 0.01
RECOPROB = 0.95      # probability of not reconstructing a real jet
AVENUMFAKES = 1      # average number of spurious (fake) recontstructions

# simulate the generator-level jets
numgenjets = numpy.random.poisson(AVENUMJETS, NUMEVENTS).astype(numpy.int32)
genstops = numpy.cumsum(numgenjets).astype(numpy.int32)
genstarts = numpy.empty_like(genstops)
genstarts[0] = 0
genstarts[1:] = genstops[:-1]
genphi = numpy.random.uniform(PHILOW, PHIHIGH, genstops[-1]).astype(numpy.float32)
geneta = numpy.random.uniform(ETALOW, ETAHIGH, genstops[-1]).astype(numpy.float32)

# simulate mismeasurement (error in reconstructing phi and eta)
phiwitherr = genphi + numpy.random.normal(0, ERRPHI, genstops[-1]).astype(numpy.float32)
etawitherr = geneta + numpy.random.normal(0, ERRETA, genstops[-1]).astype(numpy.float32)

In [3]:
# Counts per event
lengths = genstops-genstarts

In [4]:
# Cumulative number of possible combinations per event
pairs_len = numpy.zeros(len(lengths)+1, dtype=numpy.int32)
pairs_len[1:] = numpy.cumsum(lengths*lengths)

In [5]:
# Initialize deltaR matrix
deltar = numpy.empty(pairs_len[-1], dtype=numpy.float32)

In [6]:
# Calculations. First define deltaeta and deltaphi
def deltaeta(eta1, eta2):
    return eta1-eta2

def deltaphi(phi1, phi2):
    return phi1-phi2

# Okay, now that we are set up, let's calculate deltaR
for i in range(NUMEVENTS):
    pairs_i = pairs_len[i]
    for j in range(genstarts[i], genstops[i]):
        for k in range(genstarts[i], genstops[i]):
            deltar[pairs_i] = math.hypot(deltaeta(geneta[j], etawitherr[k]), deltaphi(genphi[j], phiwitherr[k]))
            pairs_i +=1

In [7]:
# Print the values
for i in range(6):
    print("Event:{} \n deltaR = {}\n".format(i, deltar[pairs_len[i]:pairs_len[i+1]]))

Event:0 
 deltaR = [  6.42889924e-03   5.74271202e+00   3.65127587e+00   3.29640198e+00
   2.54862380e+00   5.36129808e+00   5.49829865e+00   3.98654914e+00
   5.74485207e+00   4.67394479e-03   4.95202398e+00   6.84381819e+00
   6.26482677e+00   9.21387970e-01   4.73655462e+00   8.36740208e+00
   3.67095017e+00   4.96559095e+00   1.54938409e-02   6.77577305e+00
   1.96917820e+00   4.12315178e+00   1.96529949e+00   7.64784002e+00
   3.31579375e+00   6.86645508e+00   6.78617001e+00   2.57523339e-02
   5.86435509e+00   6.93174505e+00   8.42620754e+00   1.58491385e+00
   2.55180216e+00   6.26003790e+00   1.95469856e+00   5.84087849e+00
   9.59060993e-03   5.55269766e+00   3.89077353e+00   6.36314201e+00
   5.37143040e+00   9.11342382e-01   4.12801456e+00   6.90587473e+00
   5.57225037e+00   1.46205816e-02   3.82994604e+00   8.36012268e+00
   5.52243471e+00   4.75572586e+00   1.98931766e+00   8.41981697e+00
   3.91186190e+00   3.83001757e+00   1.97949763e-02   9.44731712e+00
   3.97374535e+